<a href="https://colab.research.google.com/github/AUT-Student/NN-HW5/blob/main/NN_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import prettytable
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Input, RNN
import keras.backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# Dataset

In [2]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data

Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
To: /content/lp1.data
100% 27.3k/27.3k [00:00<00:00, 1.74MB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
To: /content/lp2.data
100% 14.6k/14.6k [00:00<00:00, 2.77MB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
To: /content/lp3.data
100% 14.5k/14.5k [00:00<00:00, 746kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
To: /content/lp4.data
100% 33.6k/33.6k [00:00<00:00, 1.69MB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data
To: /content/lp5.data
100% 49.2k/49.2k [00:00<00:00, 1.07MB/s]


In [3]:
normal_label_datasets = ["normal","normal","ok","normal","normal"]

dataset_x = []
dataset_y = []
for i in range(5):
  normal_label = normal_label_datasets[i]
  dataset_i = pd.read_csv(f"/content/lp{i+1}.data", header=None)
  
  new_label = None
  new_data = None

  for j, row in dataset_i.iterrows():
    if j%16==0:
      new_data = []
      
      if row[0] == normal_label:
         new_label = 0
      else:
         new_label = 1

    else:
      new_data.append([float(item) for item in row[0].split()])

      if j%16==15:
        dataset_x.append(new_data)
        dataset_y.append(new_label)

In [4]:
dataset_x, dataset_y = shuffle(dataset_x, dataset_y, random_state=0)

In [5]:
dataset_len = len(dataset_x)

In [6]:
train_x = dataset_x[:int(0.7*dataset_len)]
train_y = dataset_y[:int(0.7*dataset_len)]

valid_x = dataset_x[int(0.7*dataset_len):int(0.8*dataset_len)]
valid_y = dataset_y[int(0.7*dataset_len):int(0.8*dataset_len)]

test_x = dataset_x[int(0.8*dataset_len):]
test_y = dataset_y[int(0.8*dataset_len):]

In [7]:
sum(train_y) * 100 / len(train_y)

71.60493827160494

In [8]:
sum(test_y) * 100 / len(test_y)

77.41935483870968

# Elman

In [9]:
class ElmanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/simple_rnn.py#L242-L492

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = units

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer="glorot_uniform", name="kernel")
      self.recurrent_kernel = self.add_weight(shape=(self.units, self.units), initializer="orthogonal", name="recurrent_kernel")
      self.bias = self.add_weight(shape=(self.units), initializer="zeros", name="bias")
      self.built = True

  def call(self, inputs, states):
      previous_hidden = states[0]
      current_hidden = K.dot(inputs, self.kernel) + K.dot(previous_hidden, self.recurrent_kernel)
      current_hidden = K.bias_add(current_hidden, self.bias)
      current_hidden = K.sigmoid(current_hidden)
      return current_hidden, [current_hidden]

In [10]:
class ElmanNetwork(keras.Model):
  def __init__(self, cell_number):
    super().__init__()
    self.model = Sequential([
                             Input((15,6,)),
                             RNN(ElmanCell(cell_number)),
                             Dense(2, activation="softmax")
                            ])
  def call(self, inputs):
    return self.model.call(inputs)

In [11]:
table = prettytable.PrettyTable()
table.field_names = ["#Cell", "#Epoch", "Train Accuracy", "Validation Accuracy", "Test Accuracy"]

for cell_number in [8, 32, 64, 128, 512, 2048]:
  elman = ElmanNetwork(cell_number)

  es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

  elman.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  history = elman.fit(x=train_x, y=train_y, epochs=250, batch_size=16, validation_data=(valid_x, valid_y), callbacks=[es_callback], verbose=0)

  table.add_row([cell_number, history.epoch[-1]+1,
                 f"{round(elman.evaluate(x=train_x, y=train_y, verbose=0)[1]*100, 2)}%",
                 f"{round(elman.evaluate(x=valid_x, y=valid_y, verbose=0)[1]*100, 2)}%",
                 f"{round(elman.evaluate(x=test_x, y=test_y, verbose=0)[1]*100, 2)}%"
                 ])
print(table)

+-------+--------+----------------+---------------------+---------------+
| #Cell | #Epoch | Train Accuracy | Validation Accuracy | Test Accuracy |
+-------+--------+----------------+---------------------+---------------+
|   8   |  141   |     86.73%     |        80.43%       |     89.25%    |
|   32  |  193   |     97.84%     |        97.83%       |     95.7%     |
|   64  |   98   |     95.68%     |        89.13%       |     90.32%    |
|  128  |   45   |     93.83%     |        93.48%       |     90.32%    |
|  512  |   29   |     89.81%     |        82.61%       |     88.17%    |
|  2048 |   46   |     99.38%     |        95.65%       |     96.77%    |
+-------+--------+----------------+---------------------+---------------+


# Jordan

In [12]:
class JordanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/simple_rnn.py#L242-L492

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = 2

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer="glorot_uniform", name="kernel")
      self.recurrent_kernel = self.add_weight(shape=(2, self.units), initializer="orthogonal", name="recurrent_kernel")
      self.bias = self.add_weight(shape=(self.units), initializer="zeros", name="bias")
      
      self.output_kernel = self.add_weight(shape=(self.units, 2), name="output_kernel")
      self.output_bias = self.add_weight(shape=(2,), initializer="zeros", name="output_bias")

      self.built = True

  def call(self, inputs, states):
      previous_output = states[0]
      current_hidden = K.dot(inputs, self.kernel) + K.dot(previous_output, self.recurrent_kernel)
      current_hidden = K.bias_add(current_hidden, self.bias)
      current_hidden = K.sigmoid(current_hidden)

      current_output = K.dot(current_hidden, self.output_kernel)
      current_output = K.bias_add(current_output, self.output_bias)
      current_output = K.softmax(current_output)

      return current_output, [current_output]

In [13]:
class JordanNetwork(keras.Model):
  def __init__(self, cell_number):
    super().__init__()
    self.model = Sequential([
                             Input((15,6,)),
                             RNN(JordanCell(cell_number)),
                            ])
  def call(self, inputs):
    return self.model.call(inputs)

In [14]:
table = prettytable.PrettyTable()
table.field_names = ["#Cell", "#Epoch", "Train Accuracy", "Validation Accuracy", "Test Accuracy"]

for cell_number in [8, 32, 64, 128, 512, 2048]:
  jordan = JordanNetwork(cell_number)

  es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

  jordan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  history = jordan.fit(x=train_x, y=train_y, epochs=250, batch_size=16, validation_data=(valid_x, valid_y), callbacks=[es_callback], verbose=0)

  table.add_row([cell_number, history.epoch[-1]+1,
                 f"{round(jordan.evaluate(x=train_x, y=train_y, verbose=0)[1]*100, 2)}%",
                 f"{round(jordan.evaluate(x=valid_x, y=valid_y, verbose=0)[1]*100, 2)}%",
                 f"{round(jordan.evaluate(x=test_x, y=test_y, verbose=0)[1]*100, 2)}%"
                 ])
print(table)

+-------+--------+----------------+---------------------+---------------+
| #Cell | #Epoch | Train Accuracy | Validation Accuracy | Test Accuracy |
+-------+--------+----------------+---------------------+---------------+
|   8   |   33   |     75.0%      |        76.09%       |     81.72%    |
|   32  |  157   |     95.37%     |        91.3%        |     94.62%    |
|   64  |   74   |     91.05%     |        73.91%       |     91.4%     |
|  128  |  118   |     97.22%     |        86.96%       |     89.25%    |
|  512  |   27   |     84.88%     |        65.22%       |     86.02%    |
|  2048 |   16   |     79.94%     |        76.09%       |     81.72%    |
+-------+--------+----------------+---------------------+---------------+
